In [17]:
import csv

with open('data_avail_june.csv', 'r') as file:
   reader = csv.DictReader(file)
   data_dict = [row for row in reader]

print(data_dict)


[{'device': 'ISKIE075199', 'isDataAvailable': 'false'}, {'device': 'ISKIE067023', 'isDataAvailable': 'true'}, {'device': 'ISKIE067024', 'isDataAvailable': 'true'}, {'device': 'ISKIE067025', 'isDataAvailable': 'true'}, {'device': 'ISKIE067026', 'isDataAvailable': 'true'}, {'device': 'ISKIE067027', 'isDataAvailable': 'true'}, {'device': 'ISKIE067028', 'isDataAvailable': 'false'}, {'device': 'ISKIE067029', 'isDataAvailable': 'false'}, {'device': 'ISKIE067030', 'isDataAvailable': 'true'}, {'device': 'ISKIE067031', 'isDataAvailable': 'true'}]


In [19]:
a = data_dict[0]
print(a)



{'device': 'ISKIE075199', 'isDataAvailable': 'false'}


In [21]:
print(a.get('device'))

ISKIE075199


In [23]:
device_june=[]
for item in data_dict:
    device_june.append(item.get('device'))

In [25]:
print(device_june)

['ISKIE075199', 'ISKIE067023', 'ISKIE067024', 'ISKIE067025', 'ISKIE067026', 'ISKIE067027', 'ISKIE067028', 'ISKIE067029', 'ISKIE067030', 'ISKIE067031']


In [33]:
for item in device_june:
    print(item)

ISKIE075199
ISKIE067023
ISKIE067024
ISKIE067025
ISKIE067026
ISKIE067027
ISKIE067028
ISKIE067029
ISKIE067030
ISKIE067031


In [10]:
import datetime
import requests
import json
import urllib3
#connection parameters
MDMadd = "https://zonos.dvc.engrid.in/zonos-api"
user = "sppl_admin"
password = "$ppli-@dm1n"
profile = "1-0:98.1.0*255"
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [12]:
import uuid
import hashlib

def create_uuid_from_string(val: str):
    hex_string = hashlib.md5(val.encode("UTF-8")).hexdigest()
    return uuid.UUID(hex=hex_string)

In [45]:
def get_profile_data(profile, devlist, from_time, to_time):
    m_values_kwh = []
    m_values_kvah = []
    for device in devlist:
#        device_id = device.get('id', 'N/A')
#        groupName = device.get('groupName', 'N/A')
        url2 = f"{MDMadd}/api/1/devices/{device}/profiles/{profile}/entries?from={from_time}&to={to_time}"
        try:
            r2 = requests.get(url2, auth=(user, password), verify=False)
#            r2.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            continue
        
        response_dict = r2.json()
        entries = response_dict.get("entries", [])
        if entries:
            first_entry = entries[0]
            metered_value = first_entry.get("meteredValues", [])

            for item in metered_value:
                if item["registerId"] == "1-0:1.8.0*255":
                    item["device"] = device
                    item["profile"] = profile
                    item["UUID"] = create_uuid_from_string(device+str(item.get('measuredAt'))+item["registerId"])
#                   item["groupName"] = groupName
                    m_values_kwh.append(item)
                elif item["registerId"] == "1-0:9.8.0*255":
                    item["device"] = device
                    item["profile"] = profile
#                    item["groupName"] = groupName
                    item["UUID"] = create_uuid_from_string(device+str(item.get('measuredAt'))+item["registerId"])
                    m_values_kvah.append(item)
                    
    
    return m_values_kwh, m_values_kvah
    

In [54]:
def get_profile_data(profile, devlist, from_time, to_time):
    m_values_kwh = []
    m_values_kvah = []
    for device in devlist:
#        device_id = device.get('id', 'N/A')
#        groupName = device.get('groupName', 'N/A')
        url2 = f"{MDMadd}/api/1/devices/{device}/profiles/{profile}/entries?from={from_time}&to={to_time}"
        try:
            r2 = requests.get(url2, auth=(user, password), verify=False)
            r2.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            continue
        
        response_dict = r2.json()
        entries = response_dict.get("entries", [])
        if entries:
            first_entry = entries[0]
            metered_value = first_entry.get("meteredValues", [])

            for item in metered_value:
                if item["registerId"] == "1-0:1.8.0*255":
                    item["device"] = device
                    item["profile"] = profile
                    item["UUID"] = create_uuid_from_string(device+str(item.get('measuredAt'))+item["registerId"])
#                   item["groupName"] = groupName
                    m_values_kwh.append(item)
                elif item["registerId"] == "1-0:9.8.0*255":
                    item["device"] = device
                    item["profile"] = profile
#                    item["groupName"] = groupName
                    item["UUID"] = create_uuid_from_string(device+str(item.get('measuredAt'))+item["registerId"])
                    m_values_kvah.append(item)
            
    
    return m_values_kwh, m_values_kvah

In [56]:
start_time_cur = "2024-06-30T18:29:00Z"
to_time_cur = "2024-06-30T18:31:00Z"
value_curr_kwh, value_curr_kvah = get_profile_data(profile, device_june, start_time_cur, to_time_cur)

Request Error: 404 Client Error: Not Found for url: https://zonos.dvc.engrid.in/zonos-api/api/1/devices/ISKIE075199/profiles/1-0:98.1.0*255/entries?from=2024-06-30T18:29:00Z&to=2024-06-30T18:31:00Z


In [49]:
import pandas as pd
df = pd.DataFrame(value_curr_kwh)
df1 = pd.DataFrame(value_curr_kvah)

print(df)
print(df1)

      registerId     value unit            measuredAt       device  \
0  1-0:1.8.0*255  4674.339  kWh  2024-06-30T18:30:00Z  ISKIE067023   
1  1-0:1.8.0*255  1351.956  kWh  2024-06-30T18:30:00Z  ISKIE067024   
2  1-0:1.8.0*255  3564.747  kWh  2024-06-30T18:30:00Z  ISKIE067025   
3  1-0:1.8.0*255  5039.384  kWh  2024-06-30T18:30:00Z  ISKIE067026   
4  1-0:1.8.0*255  2727.975  kWh  2024-06-30T18:30:00Z  ISKIE067027   
5  1-0:1.8.0*255  4455.146  kWh  2024-06-30T18:30:00Z  ISKIE067030   
6  1-0:1.8.0*255  2867.454  kWh  2024-06-30T18:30:00Z  ISKIE067031   

          profile                                  UUID  
0  1-0:98.1.0*255  dfd7a4b3-9f1e-bffe-1044-87d64920f54b  
1  1-0:98.1.0*255  ecade828-e133-5a19-1382-a424b8abc7c1  
2  1-0:98.1.0*255  f3418734-b8b9-0243-98a3-cc0bb3fea619  
3  1-0:98.1.0*255  0a5a9c5e-68b2-0582-f1dd-7ddd50b091fd  
4  1-0:98.1.0*255  71b7734e-7ebc-8527-ae8d-9960fe6ce150  
5  1-0:98.1.0*255  a9ccff00-5d8d-87d6-3a7f-ca84a8e911b8  
6  1-0:98.1.0*255  a76675df-b9a1-